In [ ]:
import numpy as np
import pandas as pd

# Чтение данных
train = pd.read_csv('../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip', header=0, delimiter="\t", quoting=3)
test = pd.read_csv('../input/word2vec-nlp-tutorial/testData.tsv.zip', header=0, delimiter="\t", quoting=3)
#print(train.shape)
#print(train["review"][0])

In [ ]:
import re
import nltk
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords

# Функция преобразования отзыва в строку слов
def review_to_words(raw_review):
    # 1. Удаление HTML
    review_text = BeautifulSoup(raw_review).get_text() 
    # 2. Удаление не букв       
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    # 3. Приведение к нижнему регистру и разбиение на слова
    words = letters_only.lower().split()                             
    # 4. Преобразование стоп слов в set для ускорения процесса
    stops = set(stopwords.words("english"))                  
    # 5. Удаление стоп слов
    meaningful_words = [w for w in words if not w in stops]   
    # 6. Соединим слова обратно в одну строку, разделенную пробелом
    return(" ".join(meaningful_words)) 

In [ ]:
# Получим количество обзоров в зависимости от размера столбца
num_reviews = train["review"].size
# Инициализируем пустой список для хранения чистых обзоров
clean_train_reviews = []
# Заполним его исходя из длины списка обзоров
for i in range(0, num_reviews):
    clean_train_reviews.append(review_to_words(train["review"][i]))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Используем CountVectorizer для "мешка слов"
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 5000) 
train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Используем RandomForestClassifier
# Обучение модели
clf = RandomForestClassifier(n_estimators = 150, random_state = 42) 
clf = clf.fit(train_data_features, train["sentiment"])

In [ ]:
# Проделаем те же преобразования для тестовых данных
num_reviews = len(test["review"])
clean_test_reviews = [] 

for i in range(0,num_reviews):
    clean_review = review_to_words(test["review"][i])
    clean_test_reviews.append(clean_review)

test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
result = clf.predict(test_data_features)

# Запист результата
output = pd.DataFrame( data={"id":test["id"], "sentiment":result})
output.to_csv("result.csv", index=False, quoting=3)